# Лабораторная работа: Мета-обучение
## Шаги 3-4: Оценка алгоритмов и построение мета-набора

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.algorithms.base_learners import BaseLearners
from src.algorithms.evaluation import AlgorithmEvaluator
from src.data.collector import OpenMLCollector

%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
# Загрузка списка датасетов
try:
    datasets_df = pd.read_csv('../data/raw/dataset_list.csv')
    print(f"Загружено {len(datasets_df)} датасетов")
except FileNotFoundError:
    print("Сначала выполните шаг 1 (сбор данных)")
    datasets_df = None

In [ ]:
if datasets_df is not None:
    # Инициализация
    learners = BaseLearners(['LogisticRegression', 'RandomForest', 'KNN'])
    evaluator = AlgorithmEvaluator(learners.get_all_algorithms())
    collector = OpenMLCollector()
    
    print(f"Алгоритмы: {learners.get_algorithm_names()}")

In [ ]:
if datasets_df is not None:
    # Тест на Iris (61)
    X, y = collector.download_dataset(61)
    if X is not None:
        scores = evaluator.evaluate_all(X, y)
        best = max(scores, key=scores.get)

        print("Результаты на Iris:")
        for algo, score in scores.items():
            print(f"  {algo}: {score:.4f}")
        print(f"\nЛучший алгоритм: {best}")

In [ ]:
if datasets_df is not None:
    # Обработка первых 50 датасетов (для демонстрации)
    sample = datasets_df.head(50)
    results = []

    for idx, row in sample.iterrows():
        X, y = collector.download_dataset(row['did'])
        if X is not None:
            scores = evaluator.evaluate_all(X, y)
            if scores:
                results.append({
                    'dataset_id': row['did'],
                    'dataset_name': row['name'],
                    'best_algorithm': max(scores, key=scores.get),
                    **{f'score_{k}': v for k, v in scores.items()}
                })

    results_df = pd.DataFrame(results)
    print(f"Обработано {len(results_df)} датасетов")
    print(f"Колонки: {list(results_df.columns)}")

In [ ]:
if 'results_df' in locals() and not results_df.empty:
    # Визуализация
    plt.figure(figsize=(10, 6))
    results_df['best_algorithm'].value_counts().plot(kind='bar', color='coral', edgecolor='black')
    plt.title('Распределение лучших алгоритмов')
    plt.xlabel('Алгоритм')
    plt.ylabel('Количество датасетов')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
    
    # Сохранение
    results_df.to_csv('../data/processed/algorithm_performance.csv', index=False)
    print("Результаты сохранены в ../data/processed/algorithm_performance.csv")